# 06-06: LINE Bot SDK使用 LINE Messaging API 创建聊天机器人。

In [ ]:
// 安装: npm install @line/bot-sdk
import * as line from '@line/bot-sdk';
import express from 'express';
// ========== 1. 基础配置 ==========
const config: line.Config = {
  channelAccessToken: process.env.LINE_CHANNEL_ACCESS_TOKEN || '',
  channelSecret: process.env.LINE_CHANNEL_SECRET || ''
};
const client = new line.Client(config);
const middleware = line.middleware(config);

In [ ]:
// ========== 2. Webhook 处理 ==========
const app = express();
app.post('/webhook', middleware, (req, res) => {
  Promise
    .all(req.body.events.map(handleEvent))
    .then(() => res.status(200).end())
    .catch((err) => {
      console.error(err);
      res.status(500).end();
    });
});
async function handleEvent(event: line.WebhookEvent): Promise<any> {
  if (event.type !== 'message' || event.message.type !== 'text') {
    return Promise.resolve(null);
  }

  const { text } = event.message;

  // 回复消息
  const echo: line.TextMessage = {
    type: 'text',
    text: `You said: ${text}`
  };

  return client.replyMessage(event.replyToken, echo);
}
// app.listen(3000);

In [ ]:
// ========== 3. 发送消息类型 ==========
async function sendMessages(userId: string) {
  // 文本消息
  await client.pushMessage(userId, {
    type: 'text',
    text: 'Hello, World!'
  });

  // 带表情
  await client.pushMessage(userId, {
    type: 'text',
    text: '$ LINE emoji $',
    emojis: [
      {
        index: 0,
        productId: '5ac1bfd5040ab15980c9b435',
        emojiId: '001'
      }
    ]
  });

  // 图片
  await client.pushMessage(userId, {
    type: 'image',
    originalContentUrl: 'https://example.com/original.jpg',
    previewImageUrl: 'https://example.com/preview.jpg'
  });

  // 视频
  await client.pushMessage(userId, {
    type: 'video',
    originalContentUrl: 'https://example.com/video.mp4',
    previewImageUrl: 'https://example.com/preview.jpg'
  });

  // 音频
  await client.pushMessage(userId, {
    type: 'audio',
    originalContentUrl: 'https://example.com/audio.mp3',
    duration: 60000
  });

  // 位置
  await client.pushMessage(userId, {
    type: 'location',
    title: 'my location',
    address: 'Tokyo, Japan',
    latitude: 35.65910807942215,
    longitude: 139.70372892916203
  });

  // 贴纸
  await client.pushMessage(userId, {
    type: 'sticker',
    packageId: '446',
    stickerId: '1988'
  });
}

In [ ]:
// ========== 4. Flex Message (富媒体) ==========
async function sendFlexMessage(userId: string) {
  const flexMessage: line.FlexMessage = {
    type: 'flex',
    altText: 'This is a Flex Message',
    contents: {
      type: 'bubble',
      hero: {
        type: 'image',
        url: 'https://scdn.line-apps.com/n/channel_devcenter/img/fx/01_1_cafe.png',
        size: 'full',
        aspectRatio: '20:13',
        aspectMode: 'cover',
        action: {
          type: 'uri',
          uri: 'http://linecorp.com/'
        }
      },
      body: {
        type: 'box',
        layout: 'vertical',
        contents: [
          {
            type: 'text',
            text: 'Brown Cafe',
            weight: 'bold',
            size: 'xl'
          },
          {
            type: 'box',
            layout: 'vertical',
            margin: 'lg',
            spacing: 'sm',
            contents: [
              {
                type: 'box',
                layout: 'baseline',
                spacing: 'sm',
                contents: [
                  {
                    type: 'text',
                    text: 'Place',
                    color: '#aaaaaa',
                    size: 'sm',
                    flex: 1
                  },
                  {
                    type: 'text',
                    text: 'Miraina Tower, 4-1-6 Shinjuku, Tokyo',
                    wrap: true,
                    color: '#666666',
                    size: 'sm',
                    flex: 5
                  }
                ]
              }
            ]
          }
        ]
      },
      footer: {
        type: 'box',
        layout: 'vertical',
        spacing: 'sm',
        contents: [
          {
            type: 'button',
            style: 'link',
            height: 'sm',
            action: {
              type: 'uri',
              label: 'CALL',
              uri: 'https://linecorp.com'
            }
          },
          {
            type: 'button',
            style: 'link',
            height: 'sm',
            action: {
              type: 'uri',
              label: 'WEBSITE',
              uri: 'https://linecorp.com'
            }
          }
        ],
        flex: 0
      }
    }
  };

  await client.pushMessage(userId, flexMessage);
}

In [ ]:
// ========== 5. Quick Reply ==========
async function sendQuickReply(userId: string) {
  const message: line.TextMessage = {
    type: 'text',
    text: 'Select your favorite food category:',
    quickReply: {
      items: [
        {
          type: 'action',
          action: {
            type: 'message',
            label: 'Sushi',
            text: 'Sushi'
          }
        },
        {
          type: 'action',
          action: {
            type: 'message',
            label: 'Tempura',
            text: 'Tempura'
          }
        },
        {
          type: 'action',
          action: {
            type: 'location',
            label: 'Send location'
          }
        }
      ]
    }
  };

  await client.pushMessage(userId, message);
}

In [ ]:
// ========== 6. Rich Menu ==========
async function createRichMenu() {
  const richMenu: line.RichMenu = {
    size: {
      width: 2500,
      height: 1686
    },
    selected: false,
    name: 'Nice richmenu',
    chatBarText: 'Tap here',
    areas: [
      {
        bounds: {
          x: 0,
          y: 0,
          width: 1250,
          height: 1686
        },
        action: {
          type: 'message',
          text: 'action1'
        }
      },
      {
        bounds: {
          x: 1250,
          y: 0,
          width: 1250,
          height: 1686
        },
        action: {
          type: 'uri',
          uri: 'https://linecorp.com'
        }
      }
    ]
  };

  const richMenuId = await client.createRichMenu(richMenu);
  console.log('Rich menu created:', richMenuId);

  // 上传图片
  // await client.setRichMenuImage(richMenuId, fs.createReadStream('./richmenu.png'));

  // 设置默认菜单
  await client.setDefaultRichMenu(richMenuId);

  // 给用户设置菜单
  // await client.linkRichMenuToUser(userId, richMenuId);

  // 删除菜单
  // await client.deleteRichMenu(richMenuId);
}